<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense,Input,Activation,BatchNormalization, Dropout
from tensorflow.keras.initializers import RandomUniform


In [ ]:
data=pd.read_csv("/content/drive/MyDrive/AppliedMachineLearning/DeepLearning/data.csv")

In [ ]:
data.shape

(20000, 3)

In [ ]:
data.columns

Index(['f1', 'f2', 'label'], dtype='object')

In [ ]:
X=data[["f1","f2"]].values
y=data["label"].values

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)


(16000, 2)
(4000, 2)


In [ ]:
print(y_train.shape)
print(y_test.shape)

(16000,)
(4000,)


In [ ]:
batch_size = 128 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler

class Metrics(tf.keras.callbacks.Callback):
  def __init__(self,validation_data):
    super().__init__()
    self.x_val=validation_data[0]
    self.y_val=validation_data[1]

  def on_train_begin(self, logs={}):
      self.history={"loss":[],"acc":[],"val_loss":[],"val_acc":[],"val_f1":[],"val_auc":[]}

  def on_epoch_end(self, epoch, logs):
      y_val_pred_proba =np.asanyarray(self.model.predict(self.x_val))
      y_val_pred=[1 if val>0.5 else 0 for val in list(y_val_pred_proba)]
      val_f1 = round(f1_score(self.y_val,y_val_pred,average='micro'), 4)
      val_auc = round(roc_auc_score(self.y_val,y_val_pred_proba), 4)
      self.history["val_f1"].append(val_f1)
      self.history["val_auc"].append(val_auc)
      self.history['loss'].append(logs.get('loss'))
      self.history['acc'].append(logs.get('accuracy'))
      self.history['val_loss'].append(logs.get('val_loss'))
      self.history['val_acc'].append(logs.get('val_accuracy'))

      loss = logs.get('loss')
      if loss is not None:
          if np.isnan(loss) or np.isinf(loss):
              print("Invalid loss and terminated at epoch {}".format(epoch))
              self.model.stop_training = True

      if epoch>0 and (logs.get("val_accuracy")>self.history["val_acc"][epoch-1]):
        self.model.save("/content/drive/MyDrive/AppliedMachineLearning/DeepLearning/model_save/epoch-{}_val_acc-{}.pkl".format(epoch,
                                                                                                                               np.round(logs.get("val_accuracy"),2)))
        

      if epoch>1 and (logs.get("val_accuracy")-self.history["val_acc"][epoch-2])<0.001:
        self.model.stop_training = True

      print("- val_f1: {} - val_auc: {}".format(val_f1,val_auc))
      return


def changeLearningRate(epoch):
  global learningrate
  if epoch>1 and metrics.history["val_acc"][epoch-1]<metrics.history["val_acc"][epoch-2] and (epoch%3)!=0:
    learningrate = learningrate*(1-0.1)
  if (epoch+1)%3==0:
    learningrate = learningrate*(1-0.05)    
  return learningrate
  

In [ ]:
%load_ext tensorboard

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(512,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(input_layer)
layer2 = Dense(256,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer1)
layer3 = Dense(128,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer2)
layer4 = Dense(64,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer3)
layer5 = Dense(32,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

learningrate=0.1
metrics=Metrics(validation_data=(X_test,y_test))
lrschedule = LearningRateScheduler(changeLearningRate,verbose=1)


model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), loss='binary_crossentropy',metrics=['accuracy'])
import datetime
log_dir="/content/drive/MyDrive/AppliedMachineLearning/DeepLearning/logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=30,batch_size=128,callbacks=[metrics,lrschedule,tensorboard_callback])

Epoch 1/30

Epoch 00001: LearningRateScheduler reducing learning rate to 0.1.
125/125 [==============================] - 2s 9ms/step - loss: 1.1174 - accuracy: 0.4958 - val_loss: 0.7152 - val_accuracy: 0.5135
- val_f1: 0.5135 - val_auc: 0.5135
Epoch 2/30

Epoch 00002: LearningRateScheduler reducing learning rate to 0.1.
125/125 [==============================] - 1s 7ms/step - loss: 0.7229 - accuracy: 0.5008 - val_loss: 0.6953 - val_accuracy: 0.5135
- val_f1: 0.5135 - val_auc: 0.5135
Epoch 3/30

Epoch 00003: LearningRateScheduler reducing learning rate to 0.095.
125/125 [==============================] - 1s 6ms/step - loss: 0.7063 - accuracy: 0.4957 - val_loss: 0.7372 - val_accuracy: 0.4865
- val_f1: 0.4865 - val_auc: 0.4865


In [ ]:
%tensorboard --logdir /content/drive/MyDrive/AppliedMachineLearning/DeepLearning/logs

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(512,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(input_layer)
layer2 = Dense(256,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer1)
layer3 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer2)
layer4 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer3)
layer5 = Dense(32,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(minval=0,maxval=1,seed=30))(layer5)
#Creating a model
model_1 = Model(inputs=input_layer,outputs=output)

learningrate=0.1
metrics=Metrics(validation_data=(X_test,y_test))
lrschedule = LearningRateScheduler(changeLearningRate,verbose=1)

model_1.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), loss='binary_crossentropy',metrics=['accuracy'])
import datetime
log_dir="/content/drive/MyDrive/AppliedMachineLearning/DeepLearning/logs_1/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
model_1.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=30,batch_size=128,callbacks=[metrics,lrschedule,tensorboard_callback])

Epoch 1/30

Epoch 00001: LearningRateScheduler reducing learning rate to 0.1.
125/125 [==============================] - 2s 9ms/step - loss: 855523.5625 - accuracy: 0.4964 - val_loss: 0.6933 - val_accuracy: 0.4913
- val_f1: 0.4912 - val_auc: 0.5
Epoch 2/30

Epoch 00002: LearningRateScheduler reducing learning rate to 0.1.
125/125 [==============================] - 1s 6ms/step - loss: 0.6938 - accuracy: 0.4991 - val_loss: 0.6940 - val_accuracy: 0.4913
- val_f1: 0.4912 - val_auc: 0.5
Epoch 3/30

Epoch 00003: LearningRateScheduler reducing learning rate to 0.095.
125/125 [==============================] - 1s 6ms/step - loss: 0.6935 - accuracy: 0.4982 - val_loss: 0.6930 - val_accuracy: 0.5088
INFO:tensorflow:Assets written to: /content/drive/MyDrive/AppliedMachineLearning/DeepLearning/model_save/epoch-2_val_acc-0.51.pkl/assets
- val_f1: 0.5088 - val_auc: 0.5
Epoch 4/30

Epoch 00004: LearningRateScheduler reducing learning rate to 0.095.
125/125 [==============================] - 1s 6ms/ste

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/AppliedMachineLearning/DeepLearning/logs_1

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(512,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(input_layer)
layer2 = Dense(256,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer1)
layer3 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer2)
layer4 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer3)
layer5 = Dense(32,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.he_uniform())(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

learningrate=0.1
metrics=Metrics(validation_data=(X_test,y_test))
lrschedule = LearningRateScheduler(changeLearningRate,verbose=1)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), loss='binary_crossentropy',metrics=['accuracy'])

import datetime
log_dir="/content/drive/MyDrive/AppliedMachineLearning/DeepLearning/logs_2/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=30,batch_size=128,callbacks=[metrics,lrschedule,tensorboard_callback])

Epoch 1/30

Epoch 00001: LearningRateScheduler reducing learning rate to 0.1.
125/125 [==============================] - 2s 8ms/step - loss: 0.6674 - accuracy: 0.5860 - val_loss: 0.6365 - val_accuracy: 0.6300
- val_f1: 0.63 - val_auc: 0.6865
Epoch 2/30

Epoch 00002: LearningRateScheduler reducing learning rate to 0.1.
125/125 [==============================] - 1s 6ms/step - loss: 0.6363 - accuracy: 0.6403 - val_loss: 0.7064 - val_accuracy: 0.5470
- val_f1: 0.547 - val_auc: 0.703
Epoch 3/30

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0855.
125/125 [==============================] - 1s 6ms/step - loss: 0.6244 - accuracy: 0.6483 - val_loss: 0.6112 - val_accuracy: 0.6605
INFO:tensorflow:Assets written to: /content/drive/MyDrive/AppliedMachineLearning/DeepLearning/model_save/epoch-2_val_acc-0.66.pkl/assets
- val_f1: 0.6605 - val_auc: 0.7346
Epoch 4/30

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0855.
125/125 [==============================] - 1s 6ms/s

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/AppliedMachineLearning/DeepLearning/logs_2

In [ ]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer
layer1 = Dense(512,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(input_layer)
layer2 = Dense(256,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer1)
layer3 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer2)
layer4 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer3)
layer5 = Dense(32,activation='relu',kernel_initializer=tf.keras.initializers.he_uniform())(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.he_uniform())(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

learningrate=0.1
metrics=Metrics(validation_data=(X_test,y_test))
lrschedule = LearningRateScheduler(changeLearningRate,verbose=1)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss='binary_crossentropy',metrics=['accuracy'])

import datetime
log_dir="/content/drive/MyDrive/AppliedMachineLearning/DeepLearning/logs_3/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)
model.fit(X_train, y_train,validation_data=(X_test,y_test),epochs=30,batch_size=128,callbacks=[metrics,lrschedule,tensorboard_callback])

Epoch 1/30

Epoch 00001: LearningRateScheduler reducing learning rate to 0.1.
125/125 [==============================] - 2s 10ms/step - loss: 7.7304 - accuracy: 0.6416 - val_loss: 0.6154 - val_accuracy: 0.6530
- val_f1: 0.653 - val_auc: 0.7322
Epoch 2/30

Epoch 00002: LearningRateScheduler reducing learning rate to 0.1.
125/125 [==============================] - 1s 8ms/step - loss: 0.6098 - accuracy: 0.6646 - val_loss: 0.6075 - val_accuracy: 0.6697
INFO:tensorflow:Assets written to: /content/drive/MyDrive/AppliedMachineLearning/DeepLearning/model_save/epoch-1_val_acc-0.67.pkl/assets
- val_f1: 0.6698 - val_auc: 0.73
Epoch 3/30

Epoch 00003: LearningRateScheduler reducing learning rate to 0.095.
125/125 [==============================] - 1s 8ms/step - loss: 0.6093 - accuracy: 0.6630 - val_loss: 0.6103 - val_accuracy: 0.6683
- val_f1: 0.6682 - val_auc: 0.7294
Epoch 4/30

Epoch 00004: LearningRateScheduler reducing learning rate to 0.095.
125/125 [==============================] - 1s 8ms/s

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/AppliedMachineLearning/DeepLearning/logs_3